In [1]:
import warnings
warnings.filterwarnings("ignore")

In [2]:
from DeepPurpose import utils, models, dataset

In [3]:
import numpy as np
import pandas as pd
import json
import os 
print(os.path.abspath('.'))

/root/data/DeepPurpose


In [4]:
affinity = pd.read_csv('affinity.txt', header=None, sep = '\t')
affinity = affinity.fillna(-1)

with open('target_seq.txt') as f:
    target = json.load(f)

with open('SMILES.txt') as f:
    drug = json.load(f)

target = list(target.values())
drug = list(drug.values())

SMILES = []
Target_seq = []
y = []

for i in range(len(drug)):
    for j in range(len(target)):
        if affinity.values[i, j] != -1:
            SMILES.append(drug[i])
            Target_seq.append(target[j])
            y.append(affinity.values[i, j])

Smile, Target, y = np.array(SMILES), np.array(Target_seq), np.array(y)
print('Done!')

Done!


In [5]:
X_drugs, X_targets, y = Smile, Target, y
print('Drug 1: ' + X_drugs[0])
print('Target 1: ' + X_targets[0])
print('Score 1: ' + str(y[0]))

Drug 1: COC1=C(C=C2C(=C1)CCN=C2C3=CC(=C(C=C3)Cl)Cl)Cl
Target 1: MTVKTEAAKGTLTYSRMRGMVAILIAFMKQRRMGLNDFIQKIANNSYACKHPEVQSILKISQPQEPELMNANPSPPPSPSQQINLGPSSNPHAKPSDFHFLKVIGKGSFGKVLLARHKAEEVFYAVKVLQKKAILKKKEEKHIMSERNVLLKNVKHPFLVGLHFSFQTADKLYFVLDYINGGELFYHLQRERCFLEPRARFYAAEIASALGYLHSLNIVYRDLKPENILLDSQGHIVLTDFGLCKENIEHNSTTSTFCGTPEYLAPEVLHKQPYDRTVDWWCLGAVLYEMLYGLPPFYSRNTAEMYDNILNKPLQLKPNITNSARHLLEGLLQKDRTKRLGAKDDFMEIKSHVFFSLINWDDLINKKITPPFNPNVSGPNDLRHFDPEFTEEPVPNSIGKSPDSVLVTASVKEAAEAFLGFSYAPPTDSFL
Score 1: 11.1


In [6]:
drug_encoding, target_encoding = 'Morgan', 'CNN'

In [7]:
train, val, test = utils.data_process(X_drugs, X_targets, y, 
                                drug_encoding, target_encoding, 
                                split_method='train_full',frac=[0.8,0.1,0.1],
                                random_seed = 1)
train.head(1)

in total: 118254 drug-target pairs
encoding drug...
unique drugs: 2068
drug encoding finished...
encoding protein...
unique target sequence: 229
protein encoding finished...
splitting dataset...
Done.


,SMILES,Target Sequence,Label,drug_encoding,target_encoding
0,COC1=C(C=C2C(=C1)CCN=C2C3=CC(=C(C=C3)Cl)Cl)Cl,MTVKTEAAKGTLTYSRMRGMVAILIAFMKQRRMGLNDFIQKIANNS...,11.1,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[M, T, V, K, T, E, A, A, K, G, T, L, T, Y, S, ..."


In [8]:
config = utils.generate_config(drug_encoding = drug_encoding, 
                         target_encoding = target_encoding, 
                         cls_hidden_dims = [1024,1024,512], 
                         train_epoch = 200, 
                         test_every_X_epoch = 10, 
                         LR = 0.001, 
                         #decay = 0.0001,
                         batch_size = 512,
                         #hidden_dim_drug = 128,
                         #mpnn_hidden_size = 128,
                         #mpnn_depth = 3, 
                         #cnn_drug_filters = [32,64,96],
                         #cnn_drug_kernels = [4,8,12],
                         cnn_target_filters = [32,64,96],
                         cnn_target_kernels = [4,8,12],
                         #transformer_n_layer_target = 2,
                         num_workers = 10
                              )

In [9]:
model = models.model_initialize(**config)

In [ ]:
model.config

In [ ]:
model.train(train, val, test)

In [11]:
model.save_model('./model_MORGAN_CNN_KIBA')